In [1]:
import pandas as pd
import numpy as np
import time, os
from collections import Counter
from scipy.linalg import norm, svd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.style
matplotlib.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 9)
%matplotlib inline

In [2]:
def rpca(X, Lpenalty=-1, Spenalty=-1, maxIter=1000):
    start = time.time()
    m, n = X.shape
    Lpenalty = Lpenalty if Lpenalty != -1 else 1
    Spenalty = Spenalty if Spenalty != -1 else 1.4 / np.sqrt(max(m,n))
    it = 0
    converged = False
    objPrev = 0.5 * squaredNorm(X)
    tol = 1e-8 * objPrev
    diff = 2 * tol
    mu = m * n / (4 * lpNorm1(X))
    
    L = np.matrix(np.zeros((m,n)))
    S = None
    E = None

    while(it < maxIter and diff > tol):
        S, l1Norm = getS(X, L, mu, Spenalty)
        L, nuclearNorm = getL(X, S, mu, Lpenalty)
        E, l2Norm = getE(X, L, S)
        obj = getObjective(nuclearNorm, l1Norm, l2Norm)
        diff = abs(objPrev - obj)
        objPrev = obj
        mu = getDynamicMu(E)
        it += 1
        if (diff < tol): converged = True
    print('Convergence: ', converged)
    print(it)
    print('Time', round(time.time() - start, 2))
    return X, L, S, E

def lpNorm1(X): 
    return abs(X).sum()

def squaredNorm(X): 
    return norm(X)**2

def getS(X, L, mu, spenalty):
    spenalty2 = spenalty * mu
    S = RcppSoftThresholdMatrix(X - L, spenalty2)
    return S, spenalty2 * lpNorm1(S)

def RcppSoftThresholdMatrix(X, penalty): 
    return np.vectorize(lambda x: RcppSoftThresholdScalar(x, penalty), otypes=[np.float])(X)
    
def RcppSoftThresholdScalar(x, penalty):
    penalized = abs(x) - penalty
    if penalized < 0: 
      return 0
    if x > 0: 
      return penalized
    return -penalized

def getL(X, S, mu, Lpenalty):
    Lpenalty2 = Lpenalty * mu
    L, Ds = SVT(X - S, Lpenalty2)
    return L, Lpenalty2 * Ds.sum()

def SVT(X, penalty):
    UDV = svd(X.T, False)
    U = np.matrix(UDV[2].T)
    V = np.matrix(UDV[0].T)
    Ds = np.diag(Dsoft(UDV[1], penalty))
    L = (U * Ds * V)
    return L, Ds
        
def Dsoft(d, penalty):
    di = np.zeros(len(d))
    for j in range(len(d)):
        penalized = d[j] - penalty
        di[j] = 0 if penalized < 0 else penalized
    return di

def getE(X, L, S):
    E = X - L - S
    return E, squaredNorm(E)

def getObjective(nuclearnorm, l1norm, l2norm): 
    return 0.5*l2norm + nuclearnorm + l1norm

def getDynamicMu(E):
    m, n = E.shape
    mu = E.std() * np.sqrt(2 * max(m,n))
    return max(mu, .01)

In [3]:
f = 7
T = 10
forcediff = True
scaled = False
sinusoidal = np.sin(2*np.pi/f * np.array(range(1,T*f+1)))
ts = sinusoidal.copy()
ts[57:60] = 100

x1 = np.insert(np.diff(ts), 0, 0) if forcediff else ts
    
if not scaled:
    mean = x1.mean()
    std = x1.std()
    x2 = (x1 - mean)/std
else:
    mean, std = 0, 1
    x2 = x1

X = np.matrix(x2).reshape(f, T, order='F')
r1 = rpca(X)
ft = lambda x: pd.DataFrame((x.T.A1 * std + mean).reshape(f, T, order='F'))
gt = lambda x: pd.DataFrame((x.T.A1 * std).reshape(f, T, order='F'))
r2 = [ft(y) for y in r1[:2]] + [gt(y) for y in r1[2:4]]

Convergence:  True
24
Time 0.03


In [4]:
#CUMSUM
init = ts[0]
dif = np.insert(np.diff(ts), 0, 0)
mean1 = dif.mean()
std1 = dif.std()
difs = (dif - mean1)/std1
mdf = np.matrix(difs).reshape(f, T, order='F')
xo = init + (mdf * std1 + mean1).T.cumsum()

In [ ]:
df = pd.Series(R[0].unstack().values) == pd.Series(R[0].values.flatten('F'))
df = pd.DataFrame(np.matrix([x.values.flatten('F') for x in R]).T, columns=['X','L','S','E'])
df['time'] = df.index

ax = df[df.S > 0].plot.scatter(x='time', y='S', color='r', label='S', s=100)
ax = df.plot.line(x='time', y='X', color='b', label='X', ax=ax)
ax = df.plot.line(x='time', y='L', color='g', label='L', ax=ax)
df.plot.line(x='time', y='E', color='k', label='E', style='--', ax=ax)
plt.show()

In [ ]:
std = np.sqrt(np.var(X.values))
mean = np.mean(X.values)

In [ ]:
def soft1(x, penalty):
    return np.sign(x) * max(abs(x) - penalty,0) if x != 0 else 0

def soft2(x, penalty):
    penalized = abs(x) - penalty
    if penalized < 0:
        return 0
    if x > 0:
        return penalized
    return -penalized

def soft3(x, penalty):
    penalized = x - penalty
    return penalized if penalized > 0 else 0

soft1(6,4), soft1(4,6), soft1(6,-4), soft1(-4,6), soft1(-6,-4), soft1(-4,-6)
soft2(6,4), soft2(4,6), soft2(6,-4), soft2(-4,6), soft2(-6,-4), soft2(-4,-6)
soft3(6,4), soft3(4,6), soft3(6,-4), soft3(-4,6), soft3(-6,-4), soft3(-4,-6)

soft1(0,4), soft1(0,6), soft1(0,-4), soft1(0,6), soft1(0,-4), soft1(0,-6)
soft3(0,4), soft3(0,6), soft3(0,-4), soft3(0,6), soft3(0,-4), soft3(0,-6)
soft1(6,4), soft1(4,6), soft1(6,-4), soft1(4,6), soft1(6,-4), soft1(4,-6)
soft3(6,4), soft3(4,6), soft3(6,-4), soft3(4,6), soft3(6,-4), soft3(4,-6)
soft1(1,4), soft1(1,6), soft1(1,-4), soft1(1,6), soft1(1,-4), soft1(1,-6)
soft3(1,4), soft3(1,6), soft3(1,-4), soft3(1,6), soft3(1,-4), soft3(1,-6)

In [ ]:
A = np.matrix([-1,2])
B = np.matrix([[1,-2],[-3,4]])
C = np.matrix([[5,3],[4,2]])
D = np.matrix([[0.68,0.597],[-0.211,0.823],[0.566,-0.605]])
E = np.matrix([[(-0.211+0.68j),(-0.605+0.823j)],[(0.597+0.566j),(0.536-0.33j)]])
F = np.matrix(np.arange(1,25).reshape(3,8))

U = np.matrix(svd(D.T, False)[2].T)
d = svd(D, False)[1]
V = np.matrix(svd(D.T, False)[0].T)
svd(B, False)[2].T == np.matrix(svd(B, False)[2]).H

In [ ]:
def Dsoft2(d, penalty):
    di = np.zeros(len(d))
    for j in range(len(d)):
        penalized = d[j] - penalty
        di[j] = 0 if penalized < 0 else penalized
    return di

def Dsoft(d, penalty):
    def debt(x):
        penalized = x - penalty
        return penalized if penalized > 0 else 0
    return map(lambda x: debt(x), d)

U = np.matrix(linalg.svd(D, False)[0])
d = np.diag(linalg.svd(D, False)[1])
V = np.matrix(linalg.svd(D, False)[2])
U * d * V

In [ ]:
x = np.array([1, 2, 4, 7, 0])
np.diff(x)
x = np.array([[1, 3, 6, 10], [0, 5, 6, 8]])
np.diff(C, axis=0)

data:  j
Dickey-Fuller = -3.3453, Lag order = 4, p-value = 0.07172
alternative hypothesis: stationary
adf$p.value > .05

from statsmodels.tsa.stattools import adfuller

k = lambda x: int(pow(len(x)-1, 1/3))
k(ts)

print(adfuller(ts, 4, 'ct', autolag=None))

x = np.random.normal(size=1000)
y = np.diff(x)
print(adfuller(x, k(x), 'c', autolag=None))

c = np.diag(C)
C1 = np.diag(c)
C1 * B

In [ ]:
from PIL import Image

np.array([16,9])*40, np.array([4,3])*120
plt.imshow(co, cmap='gray')
plt.show()

im = Image.open("/home/user/images/camera/20160704_142647_Burst34.jpg")
co = im.resize((w,h),Image.LANCZOS).convert('L').resize((480,270),Image.LANCZOS)
plt.imshow(co, cmap='gray')
plt.show()

mi = np.array(co)
im2 = Image.fromarray(mi, 'L')
plt.imshow(co, cmap='gray')
plt.show()

w, h = 160,90
1.4 / np.sqrt(max(160*90,30))

source = '/home/user/images/camera/'
res = w,h
buf = []
for i in sorted(os.listdir(source)):
    im = Image.open(source+i).convert('L').resize(res ,Image.LANCZOS)
    buf.append(np.array(im).flatten())
with open('/home/user/images/base.pic', 'w') as fo:
    for i in buf:
        fo.write('\t'.join(map(lambda x:str(x), i.tolist())) + '\n')

mn = "X", "L", "S", "E"
source = '/home/user/images/'
dest = '/burst-'
res = h, w
for i in mn:
    with open(source+'base.'+i) as fi:
        buf = []
        for j in fi:
            mi = list(map(np.uint8 , j.split('\t')))
            im = Image.fromarray(np.array(mi).reshape(res), 'L')
            buf.append(im.resize((640,360),Image.LANCZOS))
    for k in range(len(buf)):
        buf[k].save(source+'images'+i+dest+str(k)+'.gif')

Image.fromarray(buf[0].reshape(res), 'L')

In [ ]:
co = im.resize((80,45),Image.LANCZOS).convert('L')
buf = np.array(co).flatten()
mi = buf.reshape((45,80))
Image.fromarray(mi, 'L')

co = im.resize((80,45),Image.LANCZOS).convert('L')
buf = np.array(co).flatten().tolist()
mi = np.array(list(map(np.uint8 , buf))).reshape((45,80))
Image.fromarray(mi, 'L')

co = im.resize((80,45),Image.LANCZOS).convert('L')
buf = np.array(co).flatten().tolist()
di = '\t'.join(map(str, buf))
buf2 = list(map(np.uint8 , di.split('\t')))
mi = np.array(buf2).reshape((45,80))
Image.fromarray(mi, 'L')

convert -delay 50 -loop 0 imagesX/* imagesX.gif

In [ ]:
home_user = 'user'

with open('/home/{}/rpca/resources/dev_7d.tsd'.format(home_user)) as fi:
    buf = list(map(int, fi.read().split(',')))

repeat = 1
f = 1440
T = int(len(buf)/f)*repeat if (len(buf)%f == 0) else print("Incomplete Period ", len(buf)/f)
forcediff=False
scaled=False
ts = np.tile(np.array(buf), repeat)
ts[600] = 10000

x1 = np.insert(np.diff(ts), 0, 0) if forcediff else ts
    
if not scaled:
    mean = x1.mean()
    std = x1.std()
    x2 = (x1 - mean)/std
else:
    mean, std = 0, 1
    x2 = x1

X = np.matrix(x2).reshape(f, T, order='F')
spenalty = (30 / np.sqrt(max(f,T)))
print(spenalty)
r1 = rpca(X, Spenalty=spenalty, maxIter=2000)

ft = lambda x: pd.DataFrame((x.T.A1 * std + mean).reshape(f, T, order='F'))
gt = lambda x: pd.DataFrame((x.T.A1 * std).reshape(f, T, order='F'))
r2 = [ft(y) for y in r1[:2]] + [gt(y) for y in r1[2:4]]

x, l, s, e = r2

In [ ]:
cycle = int(T/repeat)
s[s != 0].iloc[:,:cycle].dropna(how='all')

In [ ]:
df = pd.DataFrame(np.matrix([x.values.flatten('F') for x in r2]).T, columns=['X','L','S','E'])
df['time'] = df.index

ax = df.plot.line(x='time', y='X', color='w', label='X')
df.plot.line(x='time', y='S', color='r', label='S', ax=ax)
plt.show()

In [ ]:
cycle = int(T/repeat)
sm = s.iloc[:,:cycle].values

cycle = int(T/repeat)
sf = s[s != 0].iloc[:,:cycle].dropna(how='all')
sm = dict([ (x,sf.loc[x][sf.loc[x].notnull()].to_dict()) for x in sf.index])

In [ ]:
d = 72
plt.plot(x.as_matrix().ravel('F')[:d])
plt.plot(l.as_matrix().ravel('F')[:d])
plt.plot(s.as_matrix().ravel('F')[:d])
plt.plot(e.as_matrix().ravel('F')[:d])
plt.show()